# Master Final Notebook

(자동 생성 파일)



---

# Diamond_final

**notebooks/Diamond_final.ipynb**


# 📚 Diamonds 데이터 분석 캡스톤

In [ ]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [ ]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print("\n===== 타입확인 =====")
print(origin.info())
origin.head()

## 과제 1. 이 데이터는 얼마나 믿을 수 있을까?

In [ ]:
origin.describe().T

#### x(길이),y(높이),z(깊이) 에서 0으로 보이는 값들이 존재 한다.

#### 비어 있는 값이 있는가?(결측치)

In [ ]:
df = origin.copy()
df.isna().sum()

#### 결측치는 존재하지 않는 것으로 보인다.

In [ ]:
for i in ["cut", "color", "clarity"]:
    print(df[i].unique())

In [ ]:
data = ["cut", "color", "clarity"]

# 1) 그래프 초기화 (밖에서 한 번만)
rows = 1
cols = len(data)
width_px = 2500
height_px = 1200
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기
for i, v in enumerate(data):
    sb.countplot(data=df, x=v, ax=ax[i])
    ax[i].grid(True)

plt.tight_layout()
plt.show()
plt.close()

cut: Ideal 범주의 빈도가 가장 많고, Fair가 가장 적다.

color: G 범주의 빈도가 가장 많고, J가 가장 적다.

clarity: I1 범주의 빈도가 가장 적으며, 상대적으로 SI1, VS2 등의 범주에 데이터가 많이 분포한다.



In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

146개의 중복 행이 검사됨. 이를 제거 후 이어서 분석 진행.

In [ ]:
# x, y, z 값이 0 이하인 데이터 비율 확인

print(
    "x, y, z 값이 0 이하인 데이터 수 :",
    df[(df["x"] <= 0) | (df["y"] <= 0) | (df["z"] <= 0)]["price"].count(),
)
print(
    "x, y, z 값이 0 이하인 데이터 비율 :",
    (df[(df["x"] <= 0) | (df["y"] <= 0) | (df["z"] <= 0)]["price"].count())
    / len(df)
    * 100,
)

x,y,z 에 관하여 0인 값들이 19개 존재하는 것으로 보여지며 0 이하의 값들은 존재 하지 않는다
다이아몬드는 물리적 실체를 가지므로 길이(x), 너비(y), 깊이(z) 중 하나라도 0인 값은 현실적으로 불가능한 기록(측정 오류)로 판단하였다. 따라서 x, y, z 중 하나라도 0인 관측치는 제거한 후 분석을 진행하였다.

In [ ]:
xyz = ["x", "y", "z"]
df1 = df[(df[xyz] != 0).all(axis=1)]
df1

In [ ]:
((df1[xyz] == 0).sum(axis=1) > 0).sum()

## 미션 2. “가격 데이터는 어떤 모습일까?”

In [ ]:
# 1) 그래프 초기화 (캔바스(fig)와 도화지(ax) 준비하기)
width_px = 1000
height_px = 800
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기 -> seaborn 사용
# (1) 히스토그램: alpha로 연하게 + 테두리 제거(또는 얇게)
sb.histplot(
    data=df1,
    x="price",
    bins=40,
    stat="density",
    color="grey",
    alpha=0.25,
    edgecolor="#000000",
    linewidth=0.15,
    ax=ax,
)

# (2) KDE: 선만 또는 fill을 아주 약하게
sb.kdeplot(
    data=df1,
    x="price",
    fill=False,  # 채우면 겹쳐서 진해질 수 있어 우선 False 추천
    linewidth=2,
    ax=ax,
)

# 3) 그래프 꾸미기
ax.grid(True)

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
# 1) 그래프 초기화 (캔바스(fig)와 도화지(ax) 준비하기)
width_px = 1000
height_px = 800
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기 -> seaborn 사용
# (1) 히스토그램: alpha로 연하게 + 테두리 제거(또는 얇게)
sb.histplot(
    data=df1,
    x="carat",
    bins=40,
    stat="density",
    color="grey",
    alpha=0.25,
    edgecolor="#000000",
    linewidth=0.15,
    ax=ax,
)

# (2) KDE: 선만 또는 fill을 아주 약하게
sb.kdeplot(
    data=df1,
    x="carat",
    fill=False,  # 채우면 겹쳐서 진해질 수 있어 우선 False 추천
    linewidth=2,
    ax=ax,
)

# 3) 그래프 꾸미기
ax.grid(True)

plt.tight_layout()
plt.show()
plt.close()

price와 carat의 분포를 확인한 결과, 두 변수 모두 낮은 값에 데이터가 몰려 있고 높은 값 방향으로 꼬리가 길게 나타났다. 이런 형태는 정규분포와는 거리가 있어 보이므로, 그대로 회귀분석을 진행하면 잔차도 한쪽으로 치우쳐 해석이 어려울 수 있다. 따라서 회귀를 바로 진행하기보다는 로그 변환처럼 분포를 완화하는 방법을 검토할 필요가 있다.

## 미션 3. “로그 변환은 왜 등장할까?”

In [ ]:
df2 = df1.copy()
df2["log_price"] = np.log(df2["price"])
df2["log_carat"] = np.log(df2["carat"])
df2.head()

In [ ]:
# 1) 그래프 초기화 (캔바스(fig)와 도화지(ax) 준비하기)
width_px = 1000
height_px = 800
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기 -> seaborn 사용
# (1) 히스토그램: alpha로 연하게 + 테두리 제거(또는 얇게)
sb.histplot(
    data=df2,
    x="log_price",
    bins=40,
    stat="density",
    color="grey",
    alpha=0.25,
    edgecolor="#000000",
    linewidth=0.15,
    ax=ax,
)

# (2) KDE: 선만 또는 fill을 아주 약하게
sb.kdeplot(
    data=df2,
    x="log_price",
    fill=False,  # 채우면 겹쳐서 진해질 수 있어 우선 False 추천
    linewidth=2,
    ax=ax,
)

# 3) 그래프 꾸미기
ax.grid(True)

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
# 1) 그래프 초기화 (캔바스(fig)와 도화지(ax) 준비하기)
width_px = 1000
height_px = 800
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기 -> seaborn 사용
# (1) 히스토그램: alpha로 연하게 + 테두리 제거(또는 얇게)
sb.histplot(
    data=df2,
    x="log_carat",
    bins=40,
    stat="density",
    color="grey",
    alpha=0.25,
    edgecolor="#000000",
    linewidth=0.15,
    ax=ax,
)

# (2) KDE: 선만 또는 fill을 아주 약하게
sb.kdeplot(
    data=df2,
    x="log_carat",
    fill=False,  # 채우면 겹쳐서 진해질 수 있어 우선 False 추천
    linewidth=2,
    ax=ax,
)

# 3) 그래프 꾸미기
ax.grid(True)

plt.tight_layout()
plt.show()
plt.close()

가격과 carat에 로그 변환을 한 결과 큰 값은 눌러서 작아지고, 작은 값은 상대적으로 덜 변해서 전체 분포가 더 균형 있게 보이게 된다.  
로그 변환 후 히스토그램을 보면 변환 전보다 오른쪽으로 길게 늘어진 꼬리가 줄어든 느낌이 있다. 즉, 극단적으로 큰 값의 영향이 줄었다고 볼 수 있다.  
다만 정규성 검정을 하면 p-value가 0에 가까워서 정규성은 충족 되지 않는다  
즉 로그 변환은 정규분포를 완벽하게 만들기보다는 치우친 분포를 덜 치우치게 만들고 극단값을 줄여서 이후 회귀분석을 더 안정적으로 하기 위해 사용한다.  

## 미션 4. “품질 등급은 가격을 얼마나 설명해 줄까?

In [ ]:
order = {
    "cut": ["Ideal", "Premium", "Very Good", "Good", "Fair"],
    "color": ["D", "E", "F", "G", "H", "I", "J"],
    "clarity": ["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1"],
}

In [ ]:
data = ["cut", "color", "clarity"]


# 1) 그래프 초기화
rows = 1
cols = len(data)
width_px = 2500
height_px = 900
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

# 2) 그래프 그리기
for i, v in enumerate(data):
    sb.boxplot(data=df2, x=v, y="log_price", order=order[v], ax=ax[i])

plt.tight_layout()
plt.show()
plt.close()

cut 기준으로는 Premium의 중앙값이 상대적으로 높고, 박스 폭도 넓어 가격 변동(분산)이 큰 편으로 보인다. Fair는 박스 폭은 비교적 좁지만 이상치가 관찰되어 일부 극단값이 존재함을 확인할 수 있다.

color는 등급 간 가격 분포가 많이 겹쳐, 색상 등급만으로 가격을 뚜렷하게 구분하기 어렵다.

clarity 역시 분포 겹침이 존재해 단독으로 가격을 완전히 가르긴 어렵지만, 세 품질 변수 중에서는 등급별 중앙값 차이가 상대적으로 더 뚜렷하게 나타나 가격 차이를 가장 잘 보여주는 변수로 관찰된다.

결론적으로, 가격 차이가 비교적 뚜렷한 품질 변수는 clarity이고, color는 등급 간 분포 겹침이 커 등급이 달라도 비슷한 가격 구간이 많이 나타난다.

## 미션 5. “Premium 컷은 정말 더 비쌀까?

독립성 t-test를 사용하여 서로 다른 두개의 그룹 간 평균의 차이가 유의미한지 여부를 판단한다.

cut에서 Premium이랑 Ideal등급인 가격정보만 따로 가져온다

In [ ]:
df3 = (
    df2.query("cut == 'Premium' or cut == 'Ideal'")[["cut", "log_price"]]
    .dropna()
    .copy()
)

premium = df3.loc[df3["cut"] == "Premium", "log_price"].reset_index(drop=True)
ideal = df3.loc[df3["cut"] == "Ideal", "log_price"].reset_index(drop=True)

df4 = concat([premium, ideal], axis=1)
df4.columns = ["Premium", "Ideal"]
df4

각각의 인덱스를 초기화 후 하나의 행으로 합쳤다

In [ ]:
data = df4[["Premium", "Ideal"]]

equal_var_fields = []
normal_dist = True
report = []

for c in data.columns:
    x = data[c].dropna()  # 결측치 제거

    s, p = normaltest(x)
    normalize = p > 0.05

    report.append({"field": c, "statistic": s, "p-value": p, "result": normalize})

    normal_dist = normal_dist and normalize
    equal_var_fields.append(x)

# 정규성 결과에 따라 등분산 검정 선택
if normal_dist:
    n = "Bartlett"
    s, p = bartlett(*equal_var_fields)
else:
    n = "Levene"
    s, p = levene(*equal_var_fields, center="median")

equal_var = p > 0.05

report.append({"field": n, "statistic": s, "p-value": p, "result": equal_var})

report_df = DataFrame(report).set_index("field")
report_df

Premium과 Ideal의 log_price에 대해 정규성 검정을 수행한 결과 두 그룹 모두 p-value≈0으로 정규성 가정이 충족되지 않았고, Levene 검정에서도 p<0.05 이므로 등분산 가정이 위배되었다.
따라서 등분산 가정 없이 평균 차이를 비교할 수 있는 Welch t-test를 적용한다.

In [ ]:
s, p = ttest_ind(
    df4["Premium"].dropna(),
    df4["Ideal"].dropna(),
    equal_var=False,
    alternative="greater",
)

result = "statistic: {0:.3f}, p-value: {1:.3f}, {2}가설 채택".format(
    s, p, "대립" if p <= 0.05 else "귀무"
)
print(result)

p = 3.79e-171로 (p<0.05) 귀무가설을 기각하였다. 따라서 Premium의 평균(log_price)은 Ideal보다 통계적으로 유의하게 더 크다

## 📝미션 6. “모든 컷은 서로 다를까?”

In [ ]:
df6 = df2[["cut", "log_price"]].dropna().copy()
df6

In [ ]:
field = "log_price"
hue_field = "cut"

df = df6[[field, hue_field]].copy()
df["idx"] = df.groupby(hue_field).cumcount()

data = pivot_table(df, index="idx", columns=hue_field, values=field, aggfunc="first")


normal_dist = True
report = []
values = []

for c in data.columns:
    target = data[c].dropna()
    values.append(target)

    s, p = normaltest(target)
    normalize = p > 0.05
    report.append({"field": c, "statistic": s, "p-value": p, "result": normalize})
    normal_dist = normal_dist and normalize

# 등분산성
if normal_dist:
    n = "Bartlett"
    s, p = bartlett(*values)
else:
    n = "Levene"
    s, p = levene(*values, center="median")

equal_var = p > 0.05
report.append({"field": n, "statistic": s, "p-value": p, "result": equal_var})

report_df = DataFrame(report).set_index("field")
report_df

cut별 log_price 분포에 대해 가정 검정을 수행한 결과, 대부분의 컷은 정규성이 충족되지 않았고, Levene 검정에서도 p=2.49e-104로 등분산성이 명확히 위배되었다.
따라서 등분산을 전제로 하는 일반 일원분산분석보다는, 등분산 가정 없이 컷 간 평균 차이를 검정할 수 있는 방법을 적용하는 것이 타당하다.

In [ ]:
anova_df = df6[[field, hue_field]]

if equal_var:
    pg_anova = anova(data=anova_df, dv=field, between=hue_field)
else:
    pg_anova = welch_anova(data=anova_df, dv=field, between=hue_field)

display(pg_anova)

f_val = pg_anova["F"].iloc[0]
p_val = pg_anova["p-unc"].iloc[0]
print(
    "statistic: {0:.3f}, p-value: {1:.3g}, {2}가설 채택".format(
        f_val, p_val, "대립" if p_val <= 0.05 else "귀무"
    )
)

if equal_var:
    print("== Tukey HSD ==")
    hoc_df = pairwise_tukey(data=anova_df, dv=field, between=hue_field)
else:
    print("== Games-Howell ==")
    hoc_df = pairwise_gameshowell(anova_df, dv=field, between=hue_field)

display(hoc_df)

Welch ANOVA 결과, cut에 따라 log_price 평균은 유의하게 달라 귀무가설을 기각하였다(p = 2.74e-223). 즉, 컷 등급에 따른 가격 차이가 전반적으로 존재함이 통계적으로 확인되었다.
평균(log_price)은 Fair(8.093) > Premium(7.950) > Good(7.842) > Very Good(7.799) > Ideal(7.639) 순으로 나타났다.
사후검정에서는 Good–Very Good만 유의한 차이가 없었으며(p=0.072), 그 외 모든 조합은 유의했다. 따라서 유의성까지 반영한 서열은 Fair > Premium > Good ≈ Very Good > Ideal로 정리된다.

## 📝미션 7. “품질 요인들은 서로 영향을 주고받을까?”

원할한 분석을 위하여 각각의 값만 따로 데이터프레임을 뽑아 분석하기 쉽게 전처리 해주었다.

In [ ]:
df7 = df2[["cut", "color", "log_price"]].copy()
df7

In [ ]:
field = "log_price"
group_field = "cut"
hue_field = "color"


data = df2[[field, group_field, hue_field]].dropna().copy()

group_values = data[group_field].unique()
group_count = len(group_values)
hue_values = data[hue_field].unique()

# 1) 그래프 초기화
rows = group_count
cols = 1
width_px = 1280 * cols
height_px = 640 * rows
figsize = (width_px / my_dpi, height_px / my_dpi)

fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)


# 2) 그래프 그리기
for i, v in enumerate(group_values):
    temp = data[data[group_field] == v]

    # KDE Plot
    sb.kdeplot(data=temp, x=field, hue=hue_field, ax=ax[i])
    ax[i].grid(True, alpha=0.3)
    ax[i].set_title("{0} = {1}".format(group_field, v), fontsize=10, pad=5)

    for h in hue_values:
        temp_sub = temp[temp[hue_field] == h]
        n = len(temp_sub[field])

        # 표본이 너무 적으면 신뢰구간 계산 불가 → 스킵
        if n < 2:
            continue

        clevel = 0.95
        dof = n - 1
        sp_mean = temp_sub[field].mean()
        sp_std = temp_sub[field].std(ddof=1)
        sp_std_error = sp_std / sqrt(n)

        # 표준오차가 0/NaN이면 스킵
        if not (sp_std_error > 0):
            continue

        cmin, cmax = t.interval(clevel, dof, loc=sp_mean, scale=sp_std_error)

        # 신뢰구간 경계선
        ax[i].axvline(cmin, linestyle=":", linewidth=0.5)
        ax[i].axvline(cmax, linestyle=":", linewidth=0.5)

        # 평균 및 신뢰구간 영역 표시
        ymin, ymax = ax[i].get_ylim()
        ax[i].fill_between([cmin, cmax], 0, ymax, alpha=0.1)
        ax[i].axvline(sp_mean, linestyle="--", linewidth=1)

        # 텍스트
        ax[i].text(
            x=(cmax - cmin) / 2 + cmin,
            y=ymax,
            s="[%s] %0.3f ~ %0.3f" % (h, cmin, cmax),
            horizontalalignment="center",
            verticalalignment="bottom",
            fontdict={"size": 6, "color": "red"},
        )

# 4) 출력
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
data = df7.copy()

xname = ["cut", "color"]
yname = "log_price"

u1 = data[xname[0]].unique()
u2 = data[xname[1]].unique()

equal_var_fields = []
normal_dist = True
report = []

for i in u1:
    for j in u2:
        filtered = data[(data[xname[0]] == i) & (data[xname[1]] == j)][yname].dropna()
        if len(filtered) == 0:
            continue

        equal_var_fields.append(filtered)

        s, p = normaltest(filtered)
        normalize = p > 0.05

        report.append(
            {
                "field": "{0}={1}, {2}={3}".format(xname[0], i, xname[1], j),
                "statistic": s,
                "p-value": p,
                "result": normalize,
            }
        )

        normal_dist = normal_dist and normalize

if normal_dist:
    n = "Bartlett"
    s, p = bartlett(*equal_var_fields)
else:
    n = "Levene"
    s, p = levene(*equal_var_fields, center="median")

equal_var = p > 0.05

report.append({"field": n, "statistic": s, "p-value": p, "result": equal_var})

report_df = DataFrame(report).set_index("field")
report_df

In [ ]:
xname = ["cut", "color"]
yname = "log_price"
data = df7[xname + [yname]]

pg_anova = anova(data=data, dv=yname, between=xname, detailed=True)
display(pg_anova)

for idx in pg_anova.index:
    row = pg_anova.iloc[idx]
    source = row["Source"]
    p_val = row["p-unc"]

    if p_val <= 0.05:
        print(f"→ {source} 효과는 유의함. (대립가설 채택: p ≤ 0.05)")
    else:
        print(f"→ {source} 효과는 유의하지 않음. (귀무가설 채택: p > 0.05)")

In [ ]:
xname = ["cut", "color"]
yname = "log_price"

data = df7[xname + [yname]]

levels = {}  # 빈 딕셔너리 만들기
# 예: {'water': 2, 'sun': 3} 이런 식 결과
for factor in xname:  # 요인을 하나씩 꺼냄
    level_count = data[factor].nunique()  # 요인의 수준 수 계산
    levels[factor] = level_count  # 딕셔너리에 저장

pvals = {}  # 빈 딕셔너리 만들기
# 예: {'water': 1.30e-20, 'sun': 2.01e-17, 'water * sun': 0.105}
for index, row in pg_anova.iterrows():  # 행을 하나씩 꺼냄
    source_name = row["Source"]  # 요인 이름
    p_value = row["p-unc"]  # 각 요인의 p-value
    pvals[source_name] = p_value  # 딕셔너리에 저장

# 주효과 사후검정
for factor in xname:
    print(f"\n[{factor}]")

    # 수준이 2개면 사후검정 필요 없음
    if levels[factor] == 2:
        print(f"- {levels[factor]}수준 요인이므로 사후검정이 필요 없습니다.")
        continue

    p_val = pvals.get(factor, None)
    if p_val is None:
        print("- ANOVA 결과에서 해당 요인을 찾을 수 없습니다.")
        continue

    if p_val > 0.05:
        print(f"- 주효과 유의하지 않음 (p={p_val:.3g}) → 사후검정 수행 안 함")
        continue

    print(f"- 주효과 유의 (p={p_val:.3g}) → 사후검정 수행")

    if equal_var:
        print(" · Tukey HSD 사용 (등분산 충족)")
        hoc_df = pairwise_tukey(data=data, dv=yname, between=factor)
    else:
        print(" · Games–Howell 사용 (등분산 불충족)")
        hoc_df = pairwise_gameshowell(data=data, dv=yname, between=factor)

    display(hoc_df)

# 상호작용 자동 처리
interaction_names = [src for src in pvals.keys() if "*" in src]

for inter in interaction_names:
    p_val = pvals[inter]
    print(f"\n[{inter}] 상호작용 효과")

    if p_val <= 0.05:
        print(f"- 상호작용 유의 (p={p_val:.3g}) → 단순주효과 분석 필요")
    else:
        print(f"- 상호작용 유의하지 않음 (p={p_val:.3g}) → 단순주효과 분석 필요 없음")

log_price(가격)에 대해 cut(컷 등급), color가 각각 어떤 영향을 미치는지와, 또 cut, clarity에 대한 두 요인이 함께 작용할 때를 검증하기 위해 이원 분산분석(2-way ANOVA)을 수행하였다.

cut * color 사후검정(Games–Howell) 결과, 평균(log_price) 차이는 대부분의 조합에서 유의했다(p≤0.05).
평균값(mean) 기준으로는 J(8.146) > I(8.023) > H(7.917) > G(7.789) > F(7.762) > D(7.617) > E(7.579) 순으로 나타난다.
따라서 color는 가격(log_price) 수준을 구분하는 데 의미가 있다.

In [ ]:
df8 = df2[["cut", "clarity", "log_price"]].copy()
df8

In [ ]:
field = "log_price"
group_field = "cut"
hue_field = "clarity"


data = df2[[field, group_field, hue_field]].dropna().copy()

group_values = data[group_field].unique()
group_count = len(group_values)
hue_values = data[hue_field].unique()

# 1) 그래프 초기화
rows = group_count
cols = 1
width_px = 1280 * cols
height_px = 640 * rows
figsize = (width_px / my_dpi, height_px / my_dpi)

fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)


# 2) 그래프 그리기
for i, v in enumerate(group_values):
    temp = data[data[group_field] == v]

    # KDE Plot
    sb.kdeplot(data=temp, x=field, hue=hue_field, ax=ax[i])
    ax[i].grid(True, alpha=0.3)
    ax[i].set_title("{0} = {1}".format(group_field, v), fontsize=10, pad=5)

    for h in hue_values:
        temp_sub = temp[temp[hue_field] == h]
        n = len(temp_sub[field])

        # 표본이 너무 적으면 신뢰구간 계산 불가 → 스킵
        if n < 2:
            continue

        clevel = 0.95
        dof = n - 1
        sp_mean = temp_sub[field].mean()
        sp_std = temp_sub[field].std(ddof=1)
        sp_std_error = sp_std / sqrt(n)

        if not (sp_std_error > 0):
            continue

        cmin, cmax = t.interval(clevel, dof, loc=sp_mean, scale=sp_std_error)

        # 신뢰구간 경계선
        ax[i].axvline(cmin, linestyle=":", linewidth=0.5)
        ax[i].axvline(cmax, linestyle=":", linewidth=0.5)

        # 평균 및 신뢰구간 영역 표시
        ymin, ymax = ax[i].get_ylim()
        ax[i].fill_between([cmin, cmax], 0, ymax, alpha=0.1)
        ax[i].axvline(sp_mean, linestyle="--", linewidth=1)

        # 텍스트
        ax[i].text(
            x=(cmax - cmin) / 2 + cmin,
            y=ymax,
            s="[%s] %0.3f ~ %0.3f" % (h, cmin, cmax),
            horizontalalignment="center",
            verticalalignment="bottom",
            fontdict={"size": 6, "color": "red"},
        )

# 4) 출력
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
xname = ["cut", "clarity"]
yname = "log_price"
data = df8[xname + [yname]]

pg_anova = anova(data=data, dv=yname, between=xname, detailed=True)
display(pg_anova)

for idx in pg_anova.index:
    row = pg_anova.iloc[idx]
    source = row["Source"]
    p_val = row["p-unc"]

    if p_val <= 0.05:
        print(f"→ {source} 효과는 유의함. (대립가설 채택: p ≤ 0.05)")
    else:
        print(f"→ {source} 효과는 유의하지 않음. (귀무가설 채택: p > 0.05)")

In [ ]:
xname = ["cut", "clarity"]
yname = "log_price"

data = df8[xname + [yname]]

levels = {}  # 빈 딕셔너리 만들기
# 예: {'water': 2, 'sun': 3} 이런 식 결과
for factor in xname:  # 요인을 하나씩 꺼냄
    level_count = data[factor].nunique()  # 요인의 수준 수 계산
    levels[factor] = level_count  # 딕셔너리에 저장

pvals = {}  # 빈 딕셔너리 만들기
# 예: {'water': 1.30e-20, 'sun': 2.01e-17, 'water * sun': 0.105}
for index, row in pg_anova.iterrows():  # 행을 하나씩 꺼냄
    source_name = row["Source"]  # 요인 이름
    p_value = row["p-unc"]  # 각 요인의 p-value
    pvals[source_name] = p_value  # 딕셔너리에 저장

# 주효과 사후검정
for factor in xname:
    print(f"\n[{factor}]")

    # 수준이 2개면 사후검정 필요 없음
    if levels[factor] == 2:
        print(f"- {levels[factor]}수준 요인이므로 사후검정이 필요 없습니다.")
        continue

    p_val = pvals.get(factor, None)
    if p_val is None:
        print("- ANOVA 결과에서 해당 요인을 찾을 수 없습니다.")
        continue

    if p_val > 0.05:
        print(f"- 주효과 유의하지 않음 (p={p_val:.3g}) → 사후검정 수행 안 함")
        continue

    print(f"- 주효과 유의 (p={p_val:.3g}) → 사후검정 수행")

    if equal_var:
        print(" · Tukey HSD 사용 (등분산 충족)")
        hoc_df = pairwise_tukey(data=data, dv=yname, between=factor)
    else:
        print(" · Games–Howell 사용 (등분산 불충족)")
        hoc_df = pairwise_gameshowell(data=data, dv=yname, between=factor)

    display(hoc_df)

# 상호작용 자동 처리
interaction_names = [src for src in pvals.keys() if "*" in src]

for inter in interaction_names:
    p_val = pvals[inter]
    print(f"\n[{inter}] 상호작용 효과")

    if p_val <= 0.05:
        print(f"- 상호작용 유의 (p={p_val:.3g}) → 단순주효과 분석 필요")
    else:
        print(f"- 상호작용 유의하지 않음 (p={p_val:.3g}) → 단순주효과 분석 필요 없음")

cut * clarity 이원분산 사후검정(Games–Howell) 결과, 평균(log_price) 차이는 대부분의 조합에서 유의하였다.(p≤0.05).
평균값(mean) 기준으로는 SI2(8.166) > I1(8.027) > SI1(7.848) > VS2(7.763) > VS1(7.725) > VVS2(7.531) > IF(7.405) > VVS1(7.322) 순으로 나타난다.
따라서 [cut × clarity] 상호작용 효과는 유의하였다(p=2.56e-49).

In [ ]:
dff = df2[["color", "clarity", "log_price"]].copy()
dff

In [ ]:
field = "log_price"
group_field = "color"
hue_field = "clarity"


data = dff[[field, group_field, hue_field]].dropna().copy()

group_values = data[group_field].unique()
group_count = len(group_values)
hue_values = data[hue_field].unique()

# 1) 그래프 초기화
rows = group_count
cols = 1
width_px = 1280 * cols
height_px = 640 * rows
figsize = (width_px / my_dpi, height_px / my_dpi)

fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)


# 2) 그래프 그리기
for i, v in enumerate(group_values):
    temp = data[data[group_field] == v]

    # KDE Plot
    sb.kdeplot(data=temp, x=field, hue=hue_field, ax=ax[i])
    ax[i].grid(True, alpha=0.3)
    ax[i].set_title("{0} = {1}".format(group_field, v), fontsize=10, pad=5)

    for h in hue_values:
        temp_sub = temp[temp[hue_field] == h]
        n = len(temp_sub[field])

        # 표본이 너무 적으면 신뢰구간 계산 불가 → 스킵
        if n < 2:
            continue

        clevel = 0.95
        dof = n - 1
        sp_mean = temp_sub[field].mean()
        sp_std = temp_sub[field].std(ddof=1)
        sp_std_error = sp_std / sqrt(n)

        if not (sp_std_error > 0):
            continue

        cmin, cmax = t.interval(clevel, dof, loc=sp_mean, scale=sp_std_error)

        # 신뢰구간 경계선
        ax[i].axvline(cmin, linestyle=":", linewidth=0.5)
        ax[i].axvline(cmax, linestyle=":", linewidth=0.5)

        # 평균 및 신뢰구간 영역 표시
        ymin, ymax = ax[i].get_ylim()
        ax[i].fill_between([cmin, cmax], 0, ymax, alpha=0.1)
        ax[i].axvline(sp_mean, linestyle="--", linewidth=1)

        # 텍스트
        ax[i].text(
            x=(cmax - cmin) / 2 + cmin,
            y=ymax,
            s="[%s] %0.3f ~ %0.3f" % (h, cmin, cmax),
            horizontalalignment="center",
            verticalalignment="bottom",
            fontdict={"size": 6, "color": "red"},
        )

# 4) 출력
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
xname = ["color", "clarity"]
yname = "log_price"
data = dff[xname + [yname]]

pg_anova = anova(data=data, dv=yname, between=xname, detailed=True)
display(pg_anova)

for idx in pg_anova.index:
    row = pg_anova.iloc[idx]
    source = row["Source"]
    p_val = row["p-unc"]

    if p_val <= 0.05:
        print(f"→ {source} 효과는 유의함. (대립가설 채택: p ≤ 0.05)")
    else:
        print(f"→ {source} 효과는 유의하지 않음. (귀무가설 채택: p > 0.05)")

In [ ]:
xname = ["color", "clarity"]
yname = "log_price"

data = dff[xname + [yname]]

levels = {}  # 빈 딕셔너리 만들기
# 예: {'water': 2, 'sun': 3} 이런 식 결과
for factor in xname:  # 요인을 하나씩 꺼냄
    level_count = data[factor].nunique()  # 요인의 수준 수 계산
    levels[factor] = level_count  # 딕셔너리에 저장

pvals = {}  # 빈 딕셔너리 만들기
# 예: {'water': 1.30e-20, 'sun': 2.01e-17, 'water * sun': 0.105}
for index, row in pg_anova.iterrows():  # 행을 하나씩 꺼냄
    source_name = row["Source"]  # 요인 이름
    p_value = row["p-unc"]  # 각 요인의 p-value
    pvals[source_name] = p_value  # 딕셔너리에 저장

# 주효과 사후검정
for factor in xname:
    print(f"\n[{factor}]")

    # 수준이 2개면 사후검정 필요 없음
    if levels[factor] == 2:
        print(f"- {levels[factor]}수준 요인이므로 사후검정이 필요 없습니다.")
        continue

    p_val = pvals.get(factor, None)
    if p_val is None:
        print("- ANOVA 결과에서 해당 요인을 찾을 수 없습니다.")
        continue

    if p_val > 0.05:
        print(f"- 주효과 유의하지 않음 (p={p_val:.3g}) → 사후검정 수행 안 함")
        continue

    print(f"- 주효과 유의 (p={p_val:.3g}) → 사후검정 수행")

    if equal_var:
        print(" · Tukey HSD 사용 (등분산 충족)")
        hoc_df = pairwise_tukey(data=data, dv=yname, between=factor)
    else:
        print(" · Games–Howell 사용 (등분산 불충족)")
        hoc_df = pairwise_gameshowell(data=data, dv=yname, between=factor)

    display(hoc_df)

# 상호작용 자동 처리
interaction_names = [src for src in pvals.keys() if "*" in src]

for inter in interaction_names:
    p_val = pvals[inter]
    print(f"\n[{inter}] 상호작용 효과")

    if p_val <= 0.05:
        print(f"- 상호작용 유의 (p={p_val:.3g}) → 단순주효과 분석 필요")
    else:
        print(f"- 상호작용 유의하지 않음 (p={p_val:.3g}) → 단순주효과 분석 필요 없음")

color * clarity 사후검정(Games–Howell) 결과, 평균(log_price) 차이는 대부분의 조합에서 유의하였다.(p≤0.05).
따라서 [color × clarity] 상호작용 효과는 유의하였다(p=1.99e-154).

이후 분석에서는 명목형변수들 사이의 교호작용을 포함하여 회귀분석을 추가적으로 진행하고자 한다

## 📝미션 8. “가격과 가장 가까운 물리적 변수는?”

In [ ]:
df8 = df2.filter(["log_price", "log_carat", "x", "y", "z", "depth", "table"])
df8

In [ ]:
skew_outlier = {}
for col in df8.columns:
    temp = df8[col].dropna()
    skew_original = temp.skew()
    z_scores = zscore(temp)
    outlier_count = int(np.sum(np.abs(z_scores) > 3))
    data_log = np.log1p(temp - temp.min() + 1)
    skew_log = data_log.skew()
    skew_outlier[col] = {
        "original_skew": skew_original,
        "log_skew": skew_log,
        "outliers(|z|>3)": outlier_count,
    }
skew_outlier_df = DataFrame(skew_outlier).T
skew_outlier_df

In [ ]:
rows = []

for a, b in combinations(df8.columns, 2):
    x = df8[a]
    y = df8[b]

    # [1] 선형성 확인 (Ramsey RESET)
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    reset = linear_reset(model, power=2, use_f=True)
    linearity_ok = reset.pvalue > 0.05

    # [2] 이상치 존재 여부(|z|>3 기준)
    a_flag = skew_outlier_df.loc[a, "outliers(|z|>3)"] > 0
    b_flag = skew_outlier_df.loc[b, "outliers(|z|>3)"] > 0
    outlier_flag = a_flag or b_flag

    # [3] 상관계수 선택 및 계산
    if linearity_ok and not outlier_flag:
        chosen = "pearson"
        corr, pval = pearsonr(x, y)
    else:
        chosen = "spearman"
        corr, pval = spearmanr(x, y)

    # [4] 유의성 판단 (p <= 0.05)
    significant = pval <= 0.05

    # [5] 상관 강도 계산
    abs_r = abs(corr)
    if abs_r >= 0.80:
        strength = "Very strong"
    elif abs_r >= 0.60:
        strength = "Strong"
    elif abs_r >= 0.40:
        strength = "Medium"
    elif abs_r >= 0.20:
        strength = "Weak"
    else:
        strength = "Very weak"

    # [6] 행 추가
    rows.append(
        {
            "var_a": a,
            "var_b": b,
            "linearity": linearity_ok,
            "outlier_flag": outlier_flag,
            "chosen": chosen,
            "corr": corr,
            "pval": pval,
            "significant": significant,
            "strength": strength,
        }
    )

summary_df = DataFrame(rows)
summary_df

In [ ]:
# 1) 등장하는 모든 변수 목록 추출
vars = sorted(set(summary_df["var_a"]).union(summary_df["var_b"]))
# 2) 빈 상관행렬 생성 (NaN으로 초기화)
corr_matrix = DataFrame(np.nan, index=vars, columns=vars)
# 3) summary_df 내용을 반영하여 대칭 행렬 채우기
for _, row in summary_df.iterrows():
    a, b, r = row["var_a"], row["var_b"], row["corr"]
    corr_matrix.loc[a, b] = r
    corr_matrix.loc[b, a] = r  # 대칭
# 4) 대각선(diagonal)을 1로 설정
np.fill_diagonal(corr_matrix.values, 1)
corr_matrix

In [ ]:
# 1) 그래프 초기화
width_px = 1000  # 그래프 가로 크기
height_px = 800  # 그래프 세로 크기
rows = 1  # 그래프 행 수
cols = 1  # 그래프 열 수
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
# 2) heatmap 그리기
sb.heatmap(
    data=corr_matrix,
    annot=True,
    fmt="0.1f",
    linewidth=0.5,
    cmap="Blues",
    annot_kws={"size": 10},
)
# 4) 출력
plt.tight_layout()  # 여백 제거
plt.show()  # 그래프 화면 출력
plt.close()  # 그래프 작업 종료

가격 분포에 왜도랑 이상치가 존재하여 Pearson보다 Spearman(순위 기반)으로 해석하였다.
그 결과 price는 x가 가장 강하게 연결되어 있었고(r=0.96), y·carat·z도 거의 비슷하게 매우 높았다.
즉 다이아가 커질수록(무게/치수 증가) 가격도 같이 올라가는 경향이 뚜렷했고, depth나 table처럼 비율 변수는 가격과 관계가 약했다.

## 미션 9. “상관관계는 곧 원인일까?
상관이 높다는 것은 무엇을 의미하는가?
이 결과를 그대로 “원인”이라고 말해도 될까?

상관이 높다는 것은 두 변수가 함께 커지거나 작아지는 경향이 강하다는 뜻이다.
하지만 상관분석은 cut/color/clarity 같은 다른 요인을 통제하지 못하므로, 이 결과를 그대로 원인이라고 단정할 수는 없다.
또한 비선형 관계, 이상치 영향 때문에 상관이 더 작게 또는 크게 보일 수도 있다.
따라서 크기 변수들이 강하게 연결돼 있다는 관찰로 해석하고, 원인 판단은 추가 검증이 필요한 영역으로 남겨두는 것이 타당하다.


---

# 원본데이터

**notebooks/jinwoo/원본데이터.ipynb**


In [ ]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [ ]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print("\n===== 타입확인 =====")
print(origin.info())
origin.head()

In [ ]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.

    Parameters
    ----------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.

    Returns
    -------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """

    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []

    for i, v in enumerate(tbl.tables[1].data):
        # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())

        def stars(p):
            return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

        # 한 변수에 대한 보고 정보 추가
        variables.append(
            {
                "종속변수": yname,  # 종속변수 이름 (외부에서 정의)
                "독립변수": name,  # 독립변수 이름
                "B": v[1].strip(),  # 비표준화 회귀계수(B)
                "표준오차": v[2].strip(),  # 계수 표준오차
                "Beta": float(fit.params[name])
                * (
                    data[name].std(ddof=1) / data[yname].std(ddof=1)
                ),  # 표준화 회귀계수(β)
                "t": "%s%s" % (v[3].strip(), stars(p)),  # t-통계량(+별표)
                "p-value": p,  # 계수 유의확률
                "공차": 1 / vif,  # 공차(Tolerance = 1/VIF)
                "vif": vif,  # 분산팽창계수
            }
        )

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for idx in [0, 2]:
        for item in tbl.tables[idx].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = (
        f"𝑅({result_dict['R-squared']}), "
        f"𝑅^2({result_dict['Adj. R-squared']}), "
        f"𝐹({result_dict['F-statistic']}), "
        f"유의확률({result_dict['Prob (F-statistic)']}), "
        f"Durbin-Watson({result_dict['Durbin-Watson']})"
    )

    # 모형 보고 문장 구성
    p_f = float(result_dict["Prob (F-statistic)"])
    sig_text = "유의하다" if p_f <= 0.05 else "유의하지 않다"
    p_sign = "≤" if p_f <= 0.05 else ">"

    tpl = (
        "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
        "이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    )

    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        sig_text,
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        p_sign,
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        p_val = float(row["p-value"])
        p_sign_var = "≤" if p_val < 0.05 else ">"
        sig_word = "유의미한" if p_val < 0.05 else "유의하지 않은"

        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                p_sign_var,
                row["종속변수"],
                sig_word,
            )
        )

    return rdf, result_report, model_report, variable_reports

In [ ]:
df = origin.filter(["price", "carat", "depth", "table", "x", "y", "z"]).copy()
df.head()

yname = "price"

x = df.drop(yname, axis=1)
y = df[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, df)
display(rdf)
print(m)

In [ ]:
while True:
    rdf, r, m, v = ols_report(tbl1, df)
    if rdf["vif"].max() >= 10:
        df = df.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = df.drop(yname, axis=1)
        y = df[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
    else:
        break

In [ ]:
display(rdf)

In [ ]:
df1 = origin.filter(["price", "carat", "cut", "color", "clarity"]).copy()

df1 = get_dummies(
    df1, columns=["cut", "color", "clarity"], dtype="int", drop_first=True
)
df1.head()

In [ ]:
yname = "price"

x = df1.drop(yname, axis=1)
y = df1[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, df1)
display(rdf)
print(m)

In [ ]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])
df2 = df1.drop(xnames, axis=1)
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl = fit.summary()
rdf, r, m, v = ols_report(tbl, df2)

In [ ]:
r


---

# 전처리

**notebooks/jinwoo/전처리.ipynb**


In [ ]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [ ]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print("\n===== 타입확인 =====")
print(origin.info())
origin.head()

In [ ]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.

    Parameters
    ----------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.

    Returns
    -------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """

    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []

    for i, v in enumerate(tbl.tables[1].data):
        # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())

        def stars(p):
            return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

        # 한 변수에 대한 보고 정보 추가
        variables.append(
            {
                "종속변수": yname,  # 종속변수 이름 (외부에서 정의)
                "독립변수": name,  # 독립변수 이름
                "B": v[1].strip(),  # 비표준화 회귀계수(B)
                "표준오차": v[2].strip(),  # 계수 표준오차
                "Beta": float(fit.params[name])
                * (
                    data[name].std(ddof=1) / data[yname].std(ddof=1)
                ),  # 표준화 회귀계수(β)
                "t": "%s%s" % (v[3].strip(), stars(p)),  # t-통계량(+별표)
                "p-value": p,  # 계수 유의확률
                "공차": 1 / vif,  # 공차(Tolerance = 1/VIF)
                "vif": vif,  # 분산팽창계수
            }
        )

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for idx in [0, 2]:
        for item in tbl.tables[idx].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = (
        f"𝑅({result_dict['R-squared']}), "
        f"𝑅^2({result_dict['Adj. R-squared']}), "
        f"𝐹({result_dict['F-statistic']}), "
        f"유의확률({result_dict['Prob (F-statistic)']}), "
        f"Durbin-Watson({result_dict['Durbin-Watson']})"
    )

    # 모형 보고 문장 구성
    p_f = float(result_dict["Prob (F-statistic)"])
    sig_text = "유의하다" if p_f <= 0.05 else "유의하지 않다"
    p_sign = "≤" if p_f <= 0.05 else ">"

    tpl = (
        "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
        "이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    )

    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        sig_text,
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        p_sign,
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        p_val = float(row["p-value"])
        p_sign_var = "≤" if p_val < 0.05 else ">"
        sig_word = "유의미한" if p_val < 0.05 else "유의하지 않은"

        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                p_sign_var,
                row["종속변수"],
                sig_word,
            )
        )

    return rdf, result_report, model_report, variable_reports

In [ ]:
xyz = ["x", "y", "z"]
origin.drop_duplicates()
df = origin[(origin[xyz] != 0).all(axis=1)]

df

In [ ]:
df = df.copy()

df["log_price"] = np.log(df["price"])
df["log_carat"] = np.log(df["carat"])
df.head()

In [ ]:
df1 = df.filter(["log_price", "log_carat", "depth", "table", "x", "y", "z"]).copy()
df1.head()

yname = "log_price"

x = df1.drop(yname, axis=1)
y = df1[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, df1)
display(rdf)
print(m)

In [ ]:
while True:
    rdf, r, m, v = ols_report(tbl1, df1)
    if rdf["vif"].max() >= 10:
        df1 = df1.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = df1.drop(yname, axis=1)
        y = df1[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
    else:
        break

In [ ]:
display(rdf)

In [ ]:
df2 = df.filter(["log_price", "log_carat", "cut", "color", "clarity"]).copy()

df2 = get_dummies(
    df2, columns=["cut", "color", "clarity"], dtype="int", drop_first=True
)
df2.head()

In [ ]:
yname = "log_price"

x = df2.drop(yname, axis=1)
y = df2[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, df2)
display(rdf)
print(m)

In [ ]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])
df3 = df2.drop(xnames, axis=1)
x = df3.drop(yname, axis=1)
y = df3[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl = fit.summary()
rdf, r, m, v = ols_report(tbl, df3)

In [ ]:
r


---

# 교효작용

**notebooks/seongyu/교효작용.ipynb**


# 1. 패키지 참조 및 데이터 가져오기

In [ ]:
from hossam import load_data
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, pearsonr, spearmanr, probplot, shapiro, kstest, anderson

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
my_dpi = 200
font_path = "./NotoSansKR-Regular.ttf"
fm.fontManager.addfont(font_path)
font_prop = fm.FontProperties(fname=font_path)
font_name = font_prop.get_name()
plt.rcParams["font.family"] = font_name
plt.rcParams["font.size"] = 6
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기 : {origin.shape}")
print(f"행 개수 : {origin.shape[0]}개")
print(f"열 개수 : {origin.shape[1]}개")
print("\n===== 타입 확인 =====")
print(origin.info())
origin.head()

# 2. 전처리 작업

In [ ]:
df = origin.drop_duplicates()
df1 = df.drop(index=df[(df["x"] == 0) | (df["y"] == 0) | (df["z"] == 0)].index)
df1.head()

In [ ]:
col_1 = "cut"
col_2 = "clarity"

df1[f"{col_1}_{col_2}"] = ""
df1 = df1.reset_index(drop=True)

for i in range(0, len(df1)):
    df1.loc[i, f"{col_1}_{col_2}"] = f"{df1.loc[i,col_1]}_{df1.loc[i,col_2]}"

df1.head()

In [ ]:
col_1 = "cut"
col_2 = "color"

df1[f"{col_1}_{col_2}"] = ""
df1 = df1.reset_index(drop=True)

for i in range(0, len(df1)):
    df1.loc[i, f"{col_1}_{col_2}"] = f"{df1.loc[i,col_1]}_{df1.loc[i,col_2]}"

df1.head()

In [ ]:
col_1 = "color"
col_2 = "clarity"

df1[f"{col_1}_{col_2}"] = ""
df1 = df1.reset_index(drop=True)

for i in range(0, len(df1)):
    df1.loc[i, f"{col_1}_{col_2}"] = f"{df1.loc[i,col_1]}_{df1.loc[i,col_2]}"

df1.head()

In [ ]:
df1["log_price"] = np.log(df1["price"])
df1["log_carat"] = np.log(df1["carat"])
df1.head()

In [ ]:
def ols_report(tbl, data):
    xnames = [n for n in fit.model.exog_names if n != "const"]
    indi_mydata1 = data.filter(xnames)
    variables = []
    for i, v in enumerate(tbl.tables[1].data):
        name = v[0].strip()
        if name not in xnames:
            continue
        j = list(indi_mydata1.columns).index(name)
        vif = variance_inflation_factor(indi_mydata1, j)

        p = float(v[4].strip())
        stars = lambda p: (
            "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
        )

        variables.append(
            {
                "종속변수": yname,
                "독립변수": name,
                "B": v[1].strip(),
                "표준오차": v[2].strip(),
                "Beta": float(fit.params[name])
                * (data[name].std(ddof=1) / data[yname].std(ddof=1)),
                "t": "%s%s" % (v[3].strip(), stars(p)),
                "p-value": p,
                "공차": 1 / vif,
                "vif": vif,
            }
        )

    rdf = DataFrame(variables)

    if rdf.empty:
        return None, "데이터 없음", "모델 유의하지 않음", []

    result_dict = {}
    for i in [0, 2]:
        for item in tbl.tables[i].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    result_report = f"R({result_dict['R-squared']}), R²({result_dict['Adj. R-squared']}), F({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})"

    tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s)=%s, p %s 0.05)."
    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        (
            "유의하다"
            if float(result_dict["Prob (F-statistic)"]) <= 0.05
            else "유의하지 않다"
        ),
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        "<=" if float(result_dict["Prob (F-statistic)"]) <= 0.05 else ">",
    )

    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                "<=" if float(row["p-value"]) < 0.05 else ">",
                row["종속변수"],
                "유의미한" if float(row["p-value"]) < 0.05 else "유의하지 않은",
            )
        )

    return rdf, result_report, model_report, variable_reports

# 3. 📊 교효작용 데이터 추가 모형

In [ ]:
mydata = df1.drop(["price", "carat"], axis=1)
yname = "log_price"
# mydata1=mydata.copy()

mydata1 = pd.get_dummies(mydata, drop_first=True)
mydata1 = mydata1.astype(float)
x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
display(rdf)
print(r)

In [ ]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])

mydata = df1.drop(["price", "carat", "x", "y", "z", "depth", "table"], axis=1)
yname = "log_price"
# mydata1=mydata.copy()

mydata1 = pd.get_dummies(mydata, drop_first=True)
mydata1 = mydata1.astype(float)
mydata1 = mydata1.drop(xnames, axis=1)
x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)

display(rdf)
print(r)

# 4. 📊 다중공선성 제거 모형

In [ ]:
mydata = df1.filter(["x", "y", "z", "depth", "table", "log_carat", "log_price"])
yname = "log_price"
mydata1 = mydata.copy()

x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print("=== 1차 회귀분석 ===")
display(rdf)
print(r)

count = 2
while True:
    rdf, r, m, v = ols_report(tbl1, mydata1)
    if rdf["vif"].max() >= 10:
        mydata1 = mydata1.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = mydata1.drop(yname, axis=1)
        y = mydata1[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
        rdf, r, m, v = ols_report(tbl1, mydata1)
        print(f"=== {count}차 회귀분석 ===")
        display(rdf)
        print(r)
        count += 1
    else:
        break

print("\n")
print("***" * 50)
print(f"총 {count} 번의 회귀분석 진행")

> z 변수보다 log_carat의 표준화계수 절대값이 높으므로, z 변수를 제거 후 명목형 변수 더미처리 이후 회귀분석 재차 진행

In [ ]:
mydata = df1.drop(["price", "carat", "x", "y", "z", "depth", "table"], axis=1)
yname = "log_price"
# mydata1=mydata.copy()

mydata1 = pd.get_dummies(mydata, drop_first=True)
mydata1 = mydata1.astype(float)
x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, mydata1)
display(rdf)
print(r)

# 5. 📊 유의하지 않은 변수 제거를 통한 최종 모형

In [ ]:
mydata = df1.filter(["x", "y", "z", "depth", "table", "log_carat", "log_price"])
yname = "log_price"
mydata1 = mydata.copy()

x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print("=== 1차 회귀분석 ===")
display(rdf)
print(r)

count = 2
while True:
    rdf, r, m, v = ols_report(tbl1, mydata1)
    if rdf["vif"].max() >= 10:
        mydata1 = mydata1.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = mydata1.drop(yname, axis=1)
        y = mydata1[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
        rdf, r, m, v = ols_report(tbl1, mydata1)
        print(f"=== {count}차 회귀분석 ===")
        display(rdf)
        print(r)
        count += 1
    else:
        break

print("\n")
print("***" * 50)
print(f"총 {count} 번의 회귀분석 진행")

In [ ]:
mydata = df1.drop(["price", "carat", "x", "y", "z", "depth", "table"], axis=1)
yname = "log_price"
# mydata1=mydata.copy()

mydata1 = pd.get_dummies(mydata, drop_first=True)
mydata1 = mydata1.astype(float)
x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, mydata1)
display(rdf)
print(r)

In [ ]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])

mydata = df1.drop(["price", "carat", "x", "y", "z", "depth", "table"], axis=1)
yname = "log_price"
# mydata1=mydata.copy()

mydata1 = pd.get_dummies(mydata, drop_first=True)
mydata1 = mydata1.astype(float)
mydata1 = mydata1.drop(xnames, axis=1)
x = mydata1.drop(yname, axis=1)
y = mydata1[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)

display(rdf)
print(r)

In [ ]:
# 잔차 검정
print("=" * 50)
print("📕 선형성 검정")
print("=" * 50)
reset_res = linear_reset(fit, power=2, use_f=True)
print(f"RESET F-statistic: {reset_res.fvalue:.4f}")
print(f"p-value: {reset_res.pvalue:.4f}")
if reset_res.pvalue < 0.05:
    print("선형성/기능형 위배 가능성이 있음 (5% 유의수준)")
else:
    print("선형성 가정 위배 근거 없음 (5% 유의수준)")

(x, y), _ = probplot(fit.resid)
width_px = 1280
height_px = 720
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.regplot(x=x, y=y, line_kws={"color": "red", "linestyle": "--", "linewidth": 2})
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

rr = fit.resid
mse_sq = fit.mse_resid
mse_root = np.sqrt(mse_sq)
colors = ["#0B3C5D", "#328CC1", "#D9EAF7"]
r1 = rr[(rr > -mse_root) & (rr < mse_root)].count() / rr.count() * 100
r2 = rr[(rr > -2 * mse_root) & (rr < 2 * mse_root)].count() / rr.count() * 100
r3 = rr[(rr > -3 * mse_root) & (rr < 3 * mse_root)].count() / rr.count() * 100
mse_r = [r1, r2, r3]
print(f"루트 1MSE 구간에 포함된 잔차 비율: {r1:.2f}%")
print(f"루트 2MSE 구간에 포함된 잔차 비율: {r2:.2f}%")
print(f"루트 3MSE 구간에 포함된 잔차 비율: {r3:.2f}%")
width_px = 1280
height_px = 720
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.residplot(x=fit.fittedvalues, y=fit.resid)
ax.grid(True, alpha=0.3)
for i, c in enumerate(colors):
    k = i + 1
    y_pos = k * mse_root
    ax.axhline(y=y_pos, color=c, linestyle="--", alpha=0.6)
    ax.axhline(y=-y_pos, color=c, linestyle="--", alpha=0.6)
    ax.text(
        x=1.02,
        y=0.5 + 0.12 * k,
        s=f"+{k} √MSE = {mse_r[i]}%",
        transform=ax.transAxes,
        ha="left",
        va="center",
        fontsize=11,
        color=c,
    )
    ax.text(
        x=1.02,
        y=0.5 - 0.12 * k,
        s=f"-{k} √MSE = {mse_r[i]}%",
        transform=ax.transAxes,
        ha="left",
        va="center",
        fontsize=11,
        color=c,
    )
plt.show()
plt.close()

# 잔차의 정규성, 등분산성, 독립성 검정
print("=" * 50)
print("🎯 잔차의 정규성, 등분산성, 독립성 검정")
print("=" * 50)
if len(origin) < 5000:
    method = "Shapiro-Wilk"
    s, p = shapiro(fit.resid)
    print(f"{method} statistic: {s:.4f}, p-value: {p:.4f}")
    violated = p < 0.05
else:
    method = "Anderson-Darling"
    ad_res = anderson(fit.resid, dist="norm")
    crit_levels = ad_res.significance_level.tolist()
    crit_values = ad_res.critical_values.tolist()
    crit_5 = crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f"{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}")
    violated = crit_5 is not None and ad_res.statistic > crit_5
print(
    "정규성 가정 위배 가능성이 있음 (5% 유의수준)"
    if violated
    else "정규성 가정 위배 근거 없음 (5% 유의수준)"
)

names = ["LM Statistic", "LM p-value", "F Statistic", "F p-value"]
bp_test = het_breuschpagan(fit.resid, fit.model.exog)
bp = DataFrame(bp_test, index=names, columns=["value"])
p_bp = bp.loc["F p-value", "value"]
print(
    f"Breusch-Pagan statistic: {bp.loc['F Statistic', 'value']:.3f}, p-value = {p_bp:.3f}"
)
print(f"{'이분산성 존재(등분산 아님)' if p_bp<=0.05 else '등분산성 만족'}")

dw = durbin_watson(fit.resid)
if 1.5 <= dw <= 2.5:
    interpretation = "잔차는 독립성을 만족함 (자기상관 없음)"
elif dw < 1.5:
    interpretation = "잔차에 양(+)의 자기상관이 존재할 가능성 있음 (독립성 위반)"
else:
    interpretation = "잔차에 음(-)의 자기상관이 존재할 가능성 있음 (독립성 위반)"
print(f"Durbin-Watson={dw:.3f}\n{interpretation}")

# 최종 결과 보고
print("=" * 50)
print("🎓 최종 결과 보고")
print("=" * 50)
print(m)
print("\n".join(v))


---

# 1사분위수_작업

**notebooks/seongyu/1사분위수_작업.ipynb**


In [ ]:
from hossam import load_data
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, pearsonr, spearmanr, probplot, shapiro, kstest, anderson

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
origin=pd.read_excel('1사분위수_데이터.xlsx')
df=origin.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
def ols_report(tbl, data):
    xnames=[n for n in fit.model.exog_names if n!='const']
    indi_mydata1=data.filter(xnames)
    variables=[]
    for i, v in enumerate(tbl.tables[1].data):
        name=v[0].strip()
        if name not in xnames:
            continue
        j=list(indi_mydata1.columns).index(name)
        vif=variance_inflation_factor(indi_mydata1,j)

        p=float(v[4].strip())
        stars=lambda p: (
            '***' if p<0.001 else
            '**' if p<0.01 else
            '*' if p<0.05 else
            ''
        )

        variables.append({
            '종속변수':yname,
            '독립변수':name,
            'B':v[1].strip(),
            '표준오차':v[2].strip(),
            'Beta':float(fit.params[name])*(data[name].std(ddof=1)/data[yname].std(ddof=1)),
            't':'%s%s'%(v[3].strip(), stars(p)),
            'p-value':p,
            '공차':1/vif,
            'vif':vif
        })

    rdf=DataFrame(variables)

    if rdf.empty:
        return None, "데이터 없음", "모델 유의하지 않음", []

    result_dict={}
    for i in [0,2]:
        for item in tbl.tables[i].data:
            n=len(item)
            for j in range(0, n, 2):
                key=item[j].strip()[:-1]
                value=item[j+1].strip()
                if not key or not value:
                    continue
                result_dict[key]=value
    
    result_report=f'R({result_dict['R-squared']}), R²({result_dict['Adj. R-squared']}), F({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})'

    tpl='%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s)=%s, p %s 0.05).'
    model_report=tpl%(
        rdf['종속변수'][0],
        ','.join(list(rdf['독립변수'])),
        '유의하다' if float(result_dict['Prob (F-statistic)'])<=0.05 else '유의하지 않다',
        result_dict['Df Model'],
        result_dict['Df Residuals'],
        result_dict['F-statistic'],
        '<=' if float(result_dict['Prob (F-statistic)'])<=0.05 else '>'
    )

    variable_reports=[]
    s='%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다.'

    for i in rdf.index:
        row=rdf.iloc[i]
        variable_reports.append(s%(row['독립변수'],
                                   row['B'],
                                   '<=' if float(row['p-value'])<0.05 else '>',
                                   row['종속변수'],
                                   '유의미한' if float(row['p-value'])<0.05 else '유의하지 않은'))
        
    return rdf, result_report, model_report, variable_reports

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.drop(['cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata1)
print('=== 1차 회귀분석 ===')
display(rdf)

> vif 값이 가장 높은 z를 제외하고 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.drop(['z','cut','color','clarity'], axis=1)
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata1)
print('=== 2차 회귀분석 ===')
display(rdf)

> vif 값이 가장 높은 x를 제외하고 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','y','depth','table'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata1)
print('=== 3차 회귀분석 ===')
display(rdf)

> vif 값이 가장 높은 y를 제외하고 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','depth','table'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata1)
print('=== 4차 회귀분석 ===')
display(rdf)

> vif 값이 가장 높은 depth를 제외하고 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','table'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata1)
print('=== 5차 회귀분석 ===')
display(rdf)

> carat과 table 모두 vif값은 동일하나, carat의 t값이 더 크기에 table을 제거 후 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
print('=== 6차 회귀분석 ===')
display(tbl1)

> 명목형 변수들을 다시 넣은 후 회귀분석 진행

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','cut','color','clarity'])
mydata2=pd.get_dummies(mydata1, drop_first=True)
mydata2=mydata2.astype(float)
x=mydata2.drop(yname, axis=1)
y=mydata2[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata2)
print('=== 7차 회귀분석 ===')
display(rdf)

> cut_Good과 cut_Very Good의 p-value 값이 0.05보다 크기 때문에 제거 후 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','cut','color','clarity'])
mydata2=pd.get_dummies(mydata1, drop_first=True)
mydata2=mydata2.astype(float)
mydata3=mydata2.drop(['cut_Good', 'cut_Very Good'], axis=1)
x=mydata3.drop(yname, axis=1)
y=mydata3[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata3)
print('=== 8차 회귀분석 ===')
display(rdf)

In [ ]:
# 잔차 검정
print('='*50)
print('📕 선형성 검정')
print('='*50)
reset_res=linear_reset(fit, power=2, use_f=True)
print(f'RESET F-statistic: {reset_res.fvalue:.4f}')
print(f'p-value: {reset_res.pvalue:.4f}')
if reset_res.pvalue<0.05:
    print('선형성/기능형 위배 가능성이 있음 (5% 유의수준)')
else:
    print('선형성 가정 위배 근거 없음 (5% 유의수준)')

(x,y),_=probplot(fit.resid)
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.regplot(x=x, y=y, line_kws={'color':'red', 'linestyle':'--', 'linewidth':2})
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

rr=fit.resid
mse_sq=fit.mse_resid
mse_root=np.sqrt(mse_sq)
colors=['#0B3C5D', '#328CC1', '#D9EAF7']
r1=rr[(rr>-mse_root)&(rr<mse_root)].count()/rr.count()*100
r2=rr[(rr>-2*mse_root)&(rr<2*mse_root)].count()/rr.count()*100
r3=rr[(rr>-3*mse_root)&(rr<3*mse_root)].count()/rr.count()*100
mse_r=[r1,r2,r3]
print(f'루트 1MSE 구간에 포함된 잔차 비율: {r1:.2f}%')
print(f'루트 2MSE 구간에 포함된 잔차 비율: {r2:.2f}%')
print(f'루트 3MSE 구간에 포함된 잔차 비율: {r3:.2f}%')
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.residplot(x=fit.fittedvalues, y=fit.resid)
ax.grid(True, alpha=0.3)
for i, c in enumerate(colors):
    k=i+1
    y_pos=k*mse_root
    ax.axhline(y=y_pos, color=c, linestyle='--', alpha=0.6)
    ax.axhline(y=-y_pos, color=c, linestyle='--', alpha=0.6)
    ax.text(x=1.02, y=0.5+0.12*k,
            s=f'+{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
    ax.text(x=1.02, y=0.5-0.12*k,
            s=f'-{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
plt.show()
plt.close()

# 잔차의 정규성, 등분산성, 독립성 검정
print('='*50)
print('🎯 잔차의 정규성, 등분산성, 독립성 검정')
print('='*50)
if len(origin)<5000:
    method='Shapiro-Wilk'
    s, p=shapiro(fit.resid)
    print(f'{method} statistic: {s:.4f}, p-value: {p:.4f}')
    violated=p<0.05
else:
    method='Anderson-Darling'
    ad_res=anderson(fit.resid, dist='norm')
    crit_levels=ad_res.significance_level.tolist()
    crit_values=ad_res.critical_values.tolist()
    crit_5=crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f'{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}')
    violated=crit_5 is not None and ad_res.statistic>crit_5
print('정규성 가정 위배 가능성이 있음 (5% 유의수준)' if violated else '정규성 가정 위배 근거 없음 (5% 유의수준)')

names=['LM Statistic', 'LM p-value', 'F Statistic', 'F p-value']
bp_test=het_breuschpagan(fit.resid, fit.model.exog)
bp=DataFrame(bp_test, index=names, columns=['value'])
p_bp=bp.loc['F p-value', 'value']
print(f'Breusch-Pagan statistic: {bp.loc['F Statistic', 'value']:.3f}, p-value = {p_bp:.3f}')
print(f'{'이분산성 존재(등분산 아님)' if p_bp<=0.05 else '등분산성 만족'}')

dw=durbin_watson(fit.resid)
if 1.5<=dw<=2.5:
    interpretation='잔차는 독립성을 만족함 (자기상관 없음)'
elif dw<1.5:
    interpretation='잔차에 양(+)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
else:
    interpretation='잔차에 음(-)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
print(f'Durbin-Watson={dw:.3f}\n{interpretation}')

# 최종 결과 보고
print('='*50)
print('🎓 최종 결과 보고')
print('='*50)
print(m)
print('\n'.join(v))

In [ ]:
r

# 아래는 참고용

⭐⭐⭐ 아래는 더미변수 모두 넣고 vif, p-value 이상치만 제거한 회귀분석 ⭐⭐⭐

In [ ]:
mydata=df
yname='price'
mydata1=mydata.copy()

mydata2=pd.get_dummies(mydata1, drop_first=True)
mydata2=mydata2.astype(float)
x=mydata2.drop(yname, axis=1)
y=mydata2[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata2)
print('=== 1차 회귀분석 ===')
display(rdf)
count=2

while True:
    if rdf['vif'].max()>=10:
        mydata2=mydata2.drop(rdf[rdf['vif']==rdf['vif'].max()]['독립변수'], axis=1)
        x=mydata2.drop(yname, axis=1)
        y=mydata2[yname]
        x_input=sm.add_constant(x)
        fit=sm.OLS(y, x_input).fit()
        tbl=fit.summary()
        rdf, r, m, v=ols_report(tbl, mydata2)
        print('=== %s차 회귀분석 ==='%count)
        display(rdf)
        count+=1
    else:
        print('***'*30)
        print('*** vif 10 이상 제거 완료 ***')
        print('***'*30)
        break

print('#'*50)
print('p-value 무효값을 제외한 최종 모델')
print('#'*50)
# p-value 무효값 제외
xnames=list(rdf[rdf['p-value']>0.05]['독립변수'])
mydata2=mydata2.drop(xnames, axis=1)
x=mydata2.drop(yname, axis=1)
y=mydata2[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl=fit.summary()
rdf, r, m, v=ols_report(tbl, mydata2)
display(rdf)

In [ ]:
# 잔차 검정
print('='*50)
print('📕 선형성 검정')
print('='*50)
reset_res=linear_reset(fit, power=2, use_f=True)
print(f'RESET F-statistic: {reset_res.fvalue:.4f}')
print(f'p-value: {reset_res.pvalue:.4f}')
if reset_res.pvalue<0.05:
    print('선형성/기능형 위배 가능성이 있음 (5% 유의수준)')
else:
    print('선형성 가정 위배 근거 없음 (5% 유의수준)')

(x,y),_=probplot(fit.resid)
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.regplot(x=x, y=y, line_kws={'color':'red', 'linestyle':'--', 'linewidth':2})
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

rr=fit.resid
mse_sq=fit.mse_resid
mse_root=np.sqrt(mse_sq)
colors=['#0B3C5D', '#328CC1', '#D9EAF7']
r1=rr[(rr>-mse_root)&(rr<mse_root)].count()/rr.count()*100
r2=rr[(rr>-2*mse_root)&(rr<2*mse_root)].count()/rr.count()*100
r3=rr[(rr>-3*mse_root)&(rr<3*mse_root)].count()/rr.count()*100
mse_r=[r1,r2,r3]
print(f'루트 1MSE 구간에 포함된 잔차 비율: {r1:.2f}%')
print(f'루트 2MSE 구간에 포함된 잔차 비율: {r2:.2f}%')
print(f'루트 3MSE 구간에 포함된 잔차 비율: {r3:.2f}%')
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.residplot(x=fit.fittedvalues, y=fit.resid)
ax.grid(True, alpha=0.3)
for i, c in enumerate(colors):
    k=i+1
    y_pos=k*mse_root
    ax.axhline(y=y_pos, color=c, linestyle='--', alpha=0.6)
    ax.axhline(y=-y_pos, color=c, linestyle='--', alpha=0.6)
    ax.text(x=1.02, y=0.5+0.12*k,
            s=f'+{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
    ax.text(x=1.02, y=0.5-0.12*k,
            s=f'-{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
plt.show()
plt.close()

# 잔차의 정규성, 등분산성, 독립성 검정
print('='*50)
print('🎯 잔차의 정규성, 등분산성, 독립성 검정')
print('='*50)
if len(origin)<5000:
    method='Shapiro-Wilk'
    s, p=shapiro(fit.resid)
    print(f'{method} statistic: {s:.4f}, p-value: {p:.4f}')
    violated=p<0.05
else:
    method='Anderson-Darling'
    ad_res=anderson(fit.resid, dist='norm')
    crit_levels=ad_res.significance_level.tolist()
    crit_values=ad_res.critical_values.tolist()
    crit_5=crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f'{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}')
    violated=crit_5 is not None and ad_res.statistic>crit_5
print('정규성 가정 위배 가능성이 있음 (5% 유의수준)' if violated else '정규성 가정 위배 근거 없음 (5% 유의수준)')

names=['LM Statistic', 'LM p-value', 'F Statistic', 'F p-value']
bp_test=het_breuschpagan(fit.resid, fit.model.exog)
bp=DataFrame(bp_test, index=names, columns=['value'])
p_bp=bp.loc['F p-value', 'value']
print(f'Breusch-Pagan statistic: {bp.loc['F Statistic', 'value']:.3f}, p-value = {p_bp:.3f}')
print(f'{'이분산성 존재(등분산 아님)' if p_bp<=0.05 else '등분산성 만족'}')

dw=durbin_watson(fit.resid)
if 1.5<=dw<=2.5:
    interpretation='잔차는 독립성을 만족함 (자기상관 없음)'
elif dw<1.5:
    interpretation='잔차에 양(+)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
else:
    interpretation='잔차에 음(-)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
print(f'Durbin-Watson={dw:.3f}\n{interpretation}')

# 최종 결과 보고
print('='*50)
print('🎓 최종 결과 보고')
print('='*50)
print(m)
print('\n'.join(v))

In [ ]:
r


---

# q2

**notebooks/jinwoo/q2.ipynb**


In [ ]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies, read_excel
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [ ]:
origin = read_excel("2사분위수_데이터.xlsx")
dfq2 = origin.drop("Unnamed: 0", axis=1)
dfq2.head()

In [ ]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.

    Parameters
    ----------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.

    Returns
    -------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """

    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []

    for i, v in enumerate(tbl.tables[1].data):
        # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())

        def stars(p):
            return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

        # 한 변수에 대한 보고 정보 추가
        variables.append(
            {
                "종속변수": yname,  # 종속변수 이름 (외부에서 정의)
                "독립변수": name,  # 독립변수 이름
                "B": v[1].strip(),  # 비표준화 회귀계수(B)
                "표준오차": v[2].strip(),  # 계수 표준오차
                "Beta": float(fit.params[name])
                * (
                    data[name].std(ddof=1) / data[yname].std(ddof=1)
                ),  # 표준화 회귀계수(β)
                "t": "%s%s" % (v[3].strip(), stars(p)),  # t-통계량(+별표)
                "p-value": p,  # 계수 유의확률
                "공차": 1 / vif,  # 공차(Tolerance = 1/VIF)
                "vif": vif,  # 분산팽창계수
            }
        )

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for idx in [0, 2]:
        for item in tbl.tables[idx].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = (
        f"𝑅({result_dict['R-squared']}), "
        f"𝑅^2({result_dict['Adj. R-squared']}), "
        f"𝐹({result_dict['F-statistic']}), "
        f"유의확률({result_dict['Prob (F-statistic)']}), "
        f"Durbin-Watson({result_dict['Durbin-Watson']})"
    )

    # 모형 보고 문장 구성
    p_f = float(result_dict["Prob (F-statistic)"])
    sig_text = "유의하다" if p_f <= 0.05 else "유의하지 않다"
    p_sign = "≤" if p_f <= 0.05 else ">"

    tpl = (
        "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
        "이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    )

    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        sig_text,
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        p_sign,
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        p_val = float(row["p-value"])
        p_sign_var = "≤" if p_val < 0.05 else ">"
        sig_word = "유의미한" if p_val < 0.05 else "유의하지 않은"

        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                p_sign_var,
                row["종속변수"],
                sig_word,
            )
        )

    return rdf, result_report, model_report, variable_reports

In [ ]:
dfq2.shape

In [ ]:
dfq2["price"].agg(["count", "min", "median", "mean", "max"])

In [ ]:
dfq2.describe().T

“Q2 구간의 표본수는 n = 13,462이며, 가격은 최소 949달러 ~ 최대 2,400달러 범위에 분포한다.”

In [ ]:
dfq2_1 = dfq2.filter(["price", "carat", "x", "y", "z", "depth", "table"]).copy()

##dfq2_1 = get_dummies(dfq2_1, columns=["cut", "color", "clarity"], dtype="int", drop_first=True)

yname = "price"

x = dfq2_1.drop(yname, axis=1)
y = dfq2_1[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, dfq2_1)
display(rdf)
print(m)

In [ ]:
dfq2_2 = dfq2_1.drop("x", axis=1)
yname = "price"
x = dfq2_2.drop(yname, axis=1)
y = dfq2_2[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, dfq2_2)
display(rdf)
print(m)

In [ ]:
dfq2_3 = dfq2_2.drop("depth", axis=1)
yname = "price"
x = dfq2_3.drop(yname, axis=1)
y = dfq2_3[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, dfq2_3)
display(rdf)
print(m)

In [ ]:
dfq2_4 = dfq2_3.drop("table", axis=1)
yname = "price"
x = dfq2_4.drop(yname, axis=1)
y = dfq2_4[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, dfq2_4)
display(rdf)
print(m)

In [ ]:
dfq2_5 = dfq2_4.drop("y", axis=1)
yname = "price"
x = dfq2_5.drop(yname, axis=1)
y = dfq2_5[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
rdf, r, m, v = ols_report(tbl1, dfq2_5)
display(rdf)
print(m)

In [ ]:
df1q2 = dfq2.filter(["price", "carat", "cut", "color", "clarity"]).copy()

df1q2 = get_dummies(
    df1q2, columns=["cut", "color", "clarity"], dtype="int", drop_first=True
)

yname = "price"

x = df1q2.drop(yname, axis=1)
y = df1q2[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

In [ ]:
rdf, r, m, v = ols_report(tbl1, df1q2)
display(rdf)
print(m)

In [ ]:
reset_res = linear_reset(fit, power=2, use_f=True)
print(f"RESET F-statistic: {reset_res.fvalue:.4f}")
print(f"p-value: {reset_res.pvalue:.4f}")
if reset_res.pvalue < 0.05:
    print("선형성/기능형 위배 가능성이 있음 (5% 유의수준)")
else:
    print("선형성 가정 위배 근거 없음 (5% 유의수준)")

In [ ]:
(x, y), _ = probplot(fit.resid)
# 1) 그래프 초기화
width_px = 1280  # 그래프 가로 크기
height_px = 720  # 그래프 세로 크기
rows = 1  # 그래프 행 수
cols = 1  # 그래프 열 수
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
# 2) regplot 그리기
sb.regplot(
    x=x, y=y, line_kws={"color": "red", "linestyle": "--", "linewidth": 2}  # 점선
)
# 3) 그래프 꾸미기
ax.grid(True, alpha=0.3)  # 배경 격자 표시
# 4) 출력
plt.tight_layout()  # 여백 제거
plt.show()  # 그래프 화면 출력
plt.close()  # 그래프 작업 종료

In [ ]:
r

In [ ]:
mse_sq = fit.mse_resid
mse_root = np.sqrt(mse_sq)
r = fit.resid  # 잔차

r1 = r[(r > -mse_root) & (r < mse_root)].count() / r.count() * 100
r2 = r[(r > -2 * mse_root) & (r < 2 * mse_root)].count() / r.count() * 100
r3 = r[(r > -3 * mse_root) & (r < 3 * mse_root)].count() / r.count() * 100

mse_r = [r1, r2, r3]

mse_r = [r1, r2, r3]
colors = ["#0B3C5D", "#328CC1", "#D9EAF7"]
xmax = fit.fittedvalues.max()
ymin = fit.fittedvalues.min()
# 1) 그래프 초기화
width_px = 1600
height_px = 720
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(1, 1, figsize=figsize, dpi=my_dpi, constrained_layout=True)
# 2) seaborn 기본 잔차도
sb.residplot(x=fit.fittedvalues, y=fit.resid)
# 3) 꾸미기
ax.grid(True, alpha=0.3)
# 4) MSE 3단계 표시
for i, c in enumerate(colors):
    k = i + 1
    # 올바른 ±k√MSE 위치
    y_pos = k * mse_root
    ax.axhline(y=y_pos, color=c, linestyle="--", alpha=0.6)
    ax.axhline(y=-y_pos, color=c, linestyle="--", alpha=0.6)
    # 텍스트는 축 좌표계 사용 (그래프 밖으로 안 나감)
    ax.text(
        x=1.02,
        y=0.5 + 0.12 * k,
        s=f"+{k} √MSE = {mse_r[i]:0.2f}%",
        transform=ax.transAxes,
        ha="left",
        va="center",
        fontsize=11,
        color=c,
    )
    ax.text(
        x=1.02,
        y=0.5 - 0.12 * k,
        s=f"-{k} √MSE = {mse_r[i]:0.2f}%",
        transform=ax.transAxes,
        ha="left",
        va="center",
        fontsize=11,
        color=c,
    )
# 5) 출력
plt.show()
plt.close()

In [ ]:
if len(df1q2) < 5000:
    method = "Shapiro-Wilk"
    stat, p = shapiro(fit.resid)
    print(f"{method} statistic: {stat:.4f}, p-value: {p:.4f}")
    violated = p < 0.05
else:
    method = "Anderson-Darling"
    ad_res = anderson(fit.resid, dist="norm")
    crit_levels = ad_res.significance_level.tolist()
    crit_values = ad_res.critical_values.tolist()
    crit_5 = crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f"{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}")
    violated = crit_5 is not None and ad_res.statistic > crit_5
print(
    "정규성 가정 위배 가능성이 있음 (5% 유의수준)"
    if violated
    else "정규성 가정 위배 근거 없음 (5%유의수준)"
)

In [ ]:
# Breusch–Pagan Test
names = ["LM Statistic", "LM p-value", "F Statistic", "F p-value"]
bp_test = het_breuschpagan(fit.resid, fit.model.exog)
bp = DataFrame(bp_test, index=names, columns=["value"])
# p-value는 F p-value를 기준으로 사용
p_bp = bp.loc["F p-value", "value"]
print(
    f"Breusch–Pagan statistic: {bp.loc['F Statistic','value']:.3f}, p-value ={p_bp:.3f}"
)
print(f"{'이분산성 존재(등분산 아님)' if p_bp <= 0.05 else '등분산성 만족'}")

In [ ]:
m

In [ ]:
print("\n".join(v))


---

# work1

**notebooks/jiyoung/work1.ipynb**


In [ ]:
from hossam import load_data
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, pearsonr, spearmanr, probplot, shapiro, kstest, anderson

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
origin = pd.read_excel('3사분위수_데이터.xlsx')
df = origin.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.
    Parameters
    ---------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.
    Returns
    ------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """
    #독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n!="const"]

    #독립변수 부분 데이터(VIF 계산용)
    indi_df = data.filter(xnames)

    #독립변수 결과를 누적
    variables = []
    for i, v in enumerate(tbl.tables[1].data):
        #한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue
        #VIF계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        #유의확률과 별표 표시 함수
        p = float(v[4].strip())
        stars = lambda p: (
            "***" if p < 0.001 else
            "**" if p < 0.01 else
            "*" if p < 0.05 else
            ""
        )

        #한 변수에 대한 보고 정보 추가
        variables.append({
            "종속변수": yname,
            "독립변수":name,
            "B":v[1].strip(),
            "표준오차":v[2].strip(),
            "Beta": float(fit.params[name]) * (data[name].std(ddof=1)/data[yname].std(ddof=1)), #표준화 회귀계수
            "t": "%s%s" % (v[3].strip(), stars(p)), #t-통계량(+별표)
            "p-value":p,
            "공차":1/vif,
            "vif":vif,
        })

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for i in[0,2]:
        for item in tbl.tables[i].data:
            n = len(item)
            for i in range(0, n, 2):
                key = item[i].strip()[:-1]
                value = item[i+1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    #적합도 보고 문자열 구성
    result_report =  f"𝑅({result_dict['R-squared']}), 𝑅^2({result_dict['Adj. R-squared']}), 𝐹({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})"
    #모형 보고 문장 구성
    tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    model_report = tpl % ( rdf['종속변수'][0],",".join(list(rdf['독립변수'])),"유의하다" if float(result_dict['Prob (F-statistic)']) <= 0.05 else "유의하지 않다",
        result_dict['Df Model'],
        result_dict['Df Residuals'],
        result_dict['F-statistic'],
        "<=" if float(result_dict['Prob (F-statistic)']) <= 0.05 else ">"
    )

    #변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        variable_reports.append(s % (row['독립변수'],row['B'],
                                     "<=" if float(row['p-value'])<0.05 else '>',
                                     row['종속변수'],
                                     '유의미한' if float(row['p-value'])<0.05 else '유의하지 않은' ))
        return rdf, result_report, model_report, variable_reports

In [ ]:
mydata=df
yname = 'price'

mydata1=mydata.drop(['cut', 'color','clarity'], axis=1)
x=mydata1.drop(yname,axis=1)
y=mydata1[yname]
x_input= sm.add_constant(x)
fit=sm.OLS(y,x_input).fit()
tbl1=fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print('=== 1차 회귀분석 ===')
display(rdf)

vif값이 가장 높은 x제거하고 다시 분석

In [ ]:
mydata=df
yname = 'price'

mydata1=mydata.drop(['x','cut', 'color','clarity'], axis=1)
x=mydata1.drop(yname,axis=1)
y=mydata1[yname]
x_input= sm.add_constant(x)
fit=sm.OLS(y,x_input).fit()
tbl1=fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print('=== 2차 회귀분석 ===')
display(rdf)

vif값이 가장 높은 z를 제거하고 다시분석

In [ ]:
mydata=df
yname = 'price'

mydata1=mydata.drop(['z','x','cut', 'color','clarity'], axis=1)
x=mydata1.drop(yname,axis=1)
y=mydata1[yname]
x_input= sm.add_constant(x)
fit=sm.OLS(y,x_input).fit()
tbl1=fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print('=== 3차 회귀분석 ===')
display(rdf)

vif값이 가장 높은 y를 제거하고 다시 분석

In [ ]:
mydata=df
yname = 'price'

mydata1=mydata.drop(['y','z','x','cut', 'color','clarity'], axis=1)
x=mydata1.drop(yname,axis=1)
y=mydata1[yname]
x_input= sm.add_constant(x)
fit=sm.OLS(y,x_input).fit()
tbl1=fit.summary()
rdf, r, m, v = ols_report(tbl1, mydata1)
print('=== 4차 회귀분석 ===')
display(rdf)

vif값이 가장 높은 table을 제외하고 다시 분석

In [ ]:
mydata=df
yname='price'

mydata1=mydata.filter(['price', 'carat','depth'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y,x_input).fit()
tbl1=fit.summary()
rdf, r, m, v= ols_report(tbl1, mydata1)
print('===5차 회귀분석')
display(rdf)

carat과 depth 모두 vif값은 동일하나, carat의 t 값이 더 크기에 depth을 제거 후 다시 분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat'])
x=mydata1.drop(yname, axis=1)
y=mydata1[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
print('=== 6차 회귀분석 ===')
display(tbl1)

명목형 변수들을 다시 넣은 후 회귀분석

In [ ]:
mydata=df
yname='price'
# mydata1=mydata.copy()

mydata1=mydata.filter(['price','carat','cut','color','clarity'])
mydata2=pd.get_dummies(mydata1, drop_first=True)
mydata2=mydata2.astype(float)
x=mydata2.drop(yname, axis=1)
y=mydata2[yname]
x_input=sm.add_constant(x)
fit=sm.OLS(y, x_input).fit()
tbl1=fit.summary()
rdf, r, m, v=ols_report(tbl1, mydata2)
print('=== 7차 회귀분석 ===')
display(rdf)

In [ ]:
# 잔차 검정
print('='*50)
print('📕 선형성 검정')
print('='*50)
reset_res=linear_reset(fit, power=2, use_f=True)
print(f'RESET F-statistic: {reset_res.fvalue:.4f}')
print(f'p-value: {reset_res.pvalue:.4f}')
if reset_res.pvalue<0.05:
    print('선형성/기능형 위배 가능성이 있음 (5% 유의수준)')
else:
    print('선형성 가정 위배 근거 없음 (5% 유의수준)')

(x,y),_=probplot(fit.resid)
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.regplot(x=x, y=y, line_kws={'color':'red', 'linestyle':'--', 'linewidth':2})
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
plt.close()

rr=fit.resid
mse_sq=fit.mse_resid
mse_root=np.sqrt(mse_sq)
colors=['#0B3C5D', '#328CC1', '#D9EAF7']
r1=rr[(rr>-mse_root)&(rr<mse_root)].count()/rr.count()*100
r2=rr[(rr>-2*mse_root)&(rr<2*mse_root)].count()/rr.count()*100
r3=rr[(rr>-3*mse_root)&(rr<3*mse_root)].count()/rr.count()*100
mse_r=[r1,r2,r3]
print(f'루트 1MSE 구간에 포함된 잔차 비율: {r1:.2f}%')
print(f'루트 2MSE 구간에 포함된 잔차 비율: {r2:.2f}%')
print(f'루트 3MSE 구간에 포함된 잔차 비율: {r3:.2f}%')
width_px=1280
height_px=720
rows=1
cols=1
figsize=(width_px/my_dpi, height_px/my_dpi)
fig, ax=plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)
sb.residplot(x=fit.fittedvalues, y=fit.resid)
ax.grid(True, alpha=0.3)
for i, c in enumerate(colors):
    k=i+1
    y_pos=k*mse_root
    ax.axhline(y=y_pos, color=c, linestyle='--', alpha=0.6)
    ax.axhline(y=-y_pos, color=c, linestyle='--', alpha=0.6)
    ax.text(x=1.02, y=0.5+0.12*k,
            s=f'+{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
    ax.text(x=1.02, y=0.5-0.12*k,
            s=f'-{k} √MSE = {mse_r[i]}%',
            transform=ax.transAxes,
            ha='left', va='center', fontsize=11, color=c)
plt.show()
plt.close()

# 잔차의 정규성, 등분산성, 독립성 검정
print('='*50)
print('🎯 잔차의 정규성, 등분산성, 독립성 검정')
print('='*50)
if len(origin)<5000:
    method='Shapiro-Wilk'
    s, p=shapiro(fit.resid)
    print(f'{method} statistic: {s:.4f}, p-value: {p:.4f}')
    violated=p<0.05
else:
    method='Anderson-Darling'
    ad_res=anderson(fit.resid, dist='norm')
    crit_levels=ad_res.significance_level.tolist()
    crit_values=ad_res.critical_values.tolist()
    crit_5=crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f'{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}')
    violated=crit_5 is not None and ad_res.statistic>crit_5
print('정규성 가정 위배 가능성이 있음 (5% 유의수준)' if violated else '정규성 가정 위배 근거 없음 (5% 유의수준)')

names=['LM Statistic', 'LM p-value', 'F Statistic', 'F p-value']
bp_test=het_breuschpagan(fit.resid, fit.model.exog)
bp=DataFrame(bp_test, index=names, columns=['value'])
p_bp=bp.loc['F p-value', 'value']
print(f'Breusch-Pagan statistic: {bp.loc['F Statistic', 'value']:.3f}, p-value = {p_bp:.3f}')
print(f'{'이분산성 존재(등분산 아님)' if p_bp<=0.05 else '등분산성 만족'}')

dw=durbin_watson(fit.resid)
if 1.5<=dw<=2.5:
    interpretation='잔차는 독립성을 만족함 (자기상관 없음)'
elif dw<1.5:
    interpretation='잔차에 양(+)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
else:
    interpretation='잔차에 음(-)의 자기상관이 존재할 가능성 있음 (독립성 위반)'
print(f'Durbin-Watson={dw:.3f}\n{interpretation}')

# 최종 결과 보고
print('='*50)
print('🎓 최종 결과 보고')
print('='*50)
print(m)
print('\n'.join(v))

p-value 값이 0.05보다 작기 때문에 잔차검정 진행

1. 연속형 변수만 선택
2. VIF 계산
4. 남은 연속형 고정
5. 명목형 더미화
6. 결합
7. 회귀
명목형 변수를 더미화할 경우 다중공선성 진단이 복잡해질 수 있어,
연속형 변수에 대해 먼저 VIF를 통해 구조적 공선성을 점검한 후
안정화된 모형에 명목형 변수를 추가하였다.


---

# 명목추가

**notebooks/sooyoung/명목추가.ipynb**


# 연속형 변수 ---> 명목형 변수 추가

In [ ]:
from pandas import DataFrame, melt
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
import pandas as pd

In [ ]:
origin = pd.read_excel('4사분위수_데이터.xlsx')
df = origin.drop('Unnamed: 0', axis=1)      # 콜론 뒤에 한 칸 띄어쓰기가 있는 경우 -> '~: 0'
df.head()

In [ ]:
df['color'].nunique()
df['color'].info()

In [ ]:
# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 1. 결과 보고서 생성

In [ ]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.
    Parameters
    ---------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.
    Returns
    ------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """
    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []
    for i, v in enumerate(tbl.tables[1].data):
    # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())
        stars = lambda p: (
            "//*" if p < 0.001 else
            "/*" if p < 0.01 else
            "*" if p < 0.05 else
            ""
        )

        # 한 변수에 대한 보고 정보 추가
        variables.append({
            "종속변수": yname,                         # 종속변수 이름
            "독립변수": name,                          # 독립변수 이름
            "B": v[1].strip(),                        # 비표준화 회귀계수(B)
            "표준오차": v[2].strip(),                 # 계수 표준오차
            "Beta": float(fit.params[name]) * (data[name].std(ddof=1) /
        data[yname].std(ddof=1)),  # 표준화 회귀계수(β)
            "t": "%s%s" % (v[3].strip(), stars(p)), # t-통계량(+별표)
            "p-value": p,                             # 계수 유의확률
            "공차": 1 / vif,                          # 공차(Tolerance = 1/VIF)
            "vif": vif,                               # 분산팽창계수
        })
        
    rdf = DataFrame(variables)

    if rdf.empty:
        return None, "데이터 없음", "모델 유의하지 않음", []

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for i in [0, 2]:
        for item in tbl.tables[i].data:
            n = len(item)
            for i in range(0, n, 2):
                key = item[i].strip()[:-1]
                value = item[i+1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = f"𝑅({result_dict['R-squared']}),𝑅^2({result_dict['Adj. R-squared']}), 𝐹({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})"

    # 모형 보고 문장 구성
    tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    model_report = tpl %(
        rdf['종속변수'][0],
        ",".join(list(rdf['독립변수'])),
        "유의하다" if float(result_dict['Prob (F-statistic)']) <= 0.05 else "유의하지 않다",
        result_dict['Df Model'],
        result_dict['Df Residuals'],
        result_dict['F-statistic'],
        "<=" if float(result_dict['Prob (F-statistic)']) <= 0.05 else ">"
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        variable_reports.append(s % (row['독립변수'],
                row['B'],
                "<=" if float(row['p-value']) < 0.05 else '>',
                row['종속변수'],
                '유의미한' if float(row['p-value']) < 0.05 else '유의하지 않은'))
        
    return rdf, result_report, model_report, variable_reports


### 2. 회귀분석

> 연속형 변수들로만 vif 10 초과를 제거 후 ->>> 명목형 변수 추가하여 p-value 확인

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.drop(['cut','color','clarity'], axis=1)     #명목형 변수 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df2)

print(' === 1차 회귀분석 ===')
display(rdf)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.drop(['x','cut','color','clarity'], axis=1)     #명목형 변수 제외 + vif_max 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df2)

print(' === 2차 회귀분석 ===')
display(rdf)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.drop(['z','x','cut','color','clarity'], axis=1)     #명목형 변수 제외 + vif_max 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df2)

print(' === 3차 회귀분석 ===')
display(rdf)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.drop(['table','z','x','cut','color','clarity'], axis=1)     #명목형 변수 제외 + vif_max 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df2)

print(' === 4차 회귀분석 ===')
display(rdf)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.drop(['y','table','z','x','cut','color','clarity'], axis=1)     #명목형 변수 제외 + vif_max 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df2)

print(' === 5차 회귀분석 ===')
display(rdf)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.filter(['price','carat'])     #명목형 변수 제외 + vif_max 제외
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

# rdf, r, m, v = ols_report(tbl1, df2)

print(' === 6차 회귀분석 ===')
display(tbl1)
print(m)

In [ ]:
df2=df.copy()
yname ='price'

df2 =df.filter(['price','carat','cut','color','clarity'])     #명목형 변수 추가
df3=pd.get_dummies(df2, drop_first=True).astype(float)
x=df3.drop(yname, axis=1)
y = df3[yname]
x_input = sm.add_constant(x)

fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()

rdf, r, m, v = ols_report(tbl1, df3)
print(' === 7차 회귀분석 ===')
display(rdf)
print(m)

### 3. 선형성 검증

In [ ]:
reset_res = linear_reset(fit, power=2, use_f=True)
print(f"RESET F-statistic: {reset_res.fvalue:.4f}")
print(f"p-value: {reset_res.pvalue:.4f}")

if reset_res.pvalue < 0.05:
    print("선형성/기능형 위배 가능성이 있음 (5% 유의수준)")
else:
    print("선형성 가정 위배 근거 없음 (5% 유의수준)")

In [ ]:
(x, y), _ = probplot(fit.resid)

width_px  = 1280
height_px = 720
rows = 1
cols = 1
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

sb.regplot(x=x, y=y, line_kws={
    "color": "red",
    "linestyle": "--",   
    "linewidth": 2
            })

ax.grid(True, alpha=0.3)   

plt.tight_layout()      
plt.show()              
plt.close()             


In [ ]:
r

In [ ]:
mse_sq = fit.mse_resid
mse_root = np.sqrt(mse_sq)
r = fit.resid   

# 잔차
r1 = r[ (r >-mse_sq) & (r < mse_sq)].count() / r.count() * 100
r2 = r[ (r >-2*mse_sq) & (r < 2*mse_sq)].count() / r.count() * 100
r3 = r[ (r >-3*mse_sq) & (r < 3*mse_sq)].count() / r.count() * 100

mse_r = [r1, r2, r3]
colors = ["#0B3C5D", "#328CC1", "#D9EAF7"]

print(f'루트 1MSE 구간에 포함된 잔차 비율: {r1:.2f}%')
print(f'루트 2MSE 구간에 포함된 잔차 비율: {r2:.2f}%')
print(f'루트 3MSE 구간에 포함된 잔차 비율: {r3:.2f}%')

xmax = fit.fittedvalues.max()
ymin = fit.fittedvalues.min()

width_px  = 1600
height_px = 720
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(1, 1, figsize=figsize, dpi=my_dpi, constrained_layout=True)

# 2) seaborn 기본 잔차도
sb.residplot(x=fit.fittedvalues, y=fit.resid)

ax.grid(True, alpha=0.3)

# 4) MSE 3단계 표시
for i, c in enumerate(colors):
    k = i + 1

    # 올바른 ±k√MSE 위치
    y_pos = k * mse_root
    ax.axhline(y= y_pos,  color=c, linestyle="--", alpha=0.6)
    ax.axhline(y=-y_pos,  color=c, linestyle="--", alpha=0.6)

    # 텍스트는 축 좌표계 사용(그래프 밖으로 안 나감)
    ax.text(x=1.02, y=0.5 + 0.12*k,
            s=f"+{k} √MSE = {mse_r[i]:0.2f}%",
            transform=ax.transAxes,
            ha="left", va="center", fontsize=11, color=c)
    ax.text(x=1.02, y=0.5 - 0.12*k,
            s=f"-{k} √MSE = {mse_r[i]:0.2f}%",
            transform=ax.transAxes,
            ha="left", va="center", fontsize=11, color=c)

plt.show()
plt.close()


In [ ]:
print('===== 잔차의 정규성 검정 =====')

if len(origin) < 5000:
    method = "Shapiro-Wilk"
    stat, p = shapiro(fit.resid)
    print(f"{method} statistic: {stat:.4f}, p-value: {p:.4f}")
    violated = p < 0.05
else:
    method = "Anderson-Darling"
    ad_res = anderson(fit.resid, dist='norm')
    crit_levels = ad_res.significance_level.tolist()
    crit_values = ad_res.critical_values.tolist()
    crit_5 = crit_values[crit_levels.index(5.0)] if 5.0 in crit_levels else None
    print(f"{method} statistic: {ad_res.statistic:.4f}, 5% 임계값: {crit_5:.4f}")
    violated = crit_5 is not None and ad_res.statistic > crit_5

print("정규성 가정 위배 가능성이 있음 (5% 유의수준)" if violated else "정규성 가정 위배 근거 없음 (5% 유의수준)")


print('\n===== 잔차의 등분산성  =====')
# Breusch-pegan Test
names = ['LM Statistic', 'LM p-value', 'F Statistic', 'F p-value']
bp_test = het_breuschpagan(fit.resid, fit.model.exog)
bp = DataFrame(bp_test, index=names, columns=['value'])

# p-value는 F p-value를 기준으로 사용
p_bp = bp.loc['F p-value', 'value']

print(f"Breusch–Pagan statistic: {bp.loc['F Statistic','value']:.3f}, p-value = {p_bp:.3f}")
print(f"{'이분산성 존재(등분산 아님)' if p_bp <= 0.05 else '등분산성 만족'}")


print(" \n===== 잔차의 독립성 =====")

dw = durbin_watson(fit.resid)

if 1.5 <= dw <= 2.5:
    interpretation = "잔차는 독립성을 만족함 (자기상관 없음)"
elif dw < 1.5:
    interpretation = "잔차에 양(+)의 자기상관이 존재할 가능성 있음 (독립성 위반)"
else:
    interpretation = "잔차에 음(–)의 자기상관이 존재할 가능성 있음 (독립성 위반)"

print(f"Durbin–Watson: {dw:.3f}")
print(interpretation)

print("\n ====== 결과 보고 ======")
m           # 모형 적합도
print("\n".join(v))
